In [1]:
from google.colab import drive
drive.mount('/content/drive')

# 주요 라이브러리 설치
!pip install mxnet
!pip install gluonnlp
!pip install transformers
!pip install sentencepiece

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 33.5 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680538 sha256=4882546a927bf798f49e829ecf3b2e77ea7a4dc2c8b11820fbddb11123692678
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━

In [2]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# seed 값 설정
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import transformers
from transformers import AdamW

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [3]:
from transformers import BertTokenizerFast, BertForSequenceClassification
# # 만약 colab pro가 아니면 model_name = 'bert-base-uncased'를 사용하세요
model_name = 'bert-base-uncased' #"bert-large-uncased"

tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [4]:
# device 설정
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cuda:0


In [5]:
# 데이터 불러오기
path = '/content/drive/MyDrive/2023/AIVLE/NLP미프/2023.04.03_미니프로젝트4차_실습자료' #'/content/drive/MyDrive/데이콘 뉴스 분류/data'
train = pd.read_csv(path+"/train.csv")

test = pd.read_csv(path+"/test.csv")
submission = pd.read_csv(path+"/random_submission.csv")

In [6]:
train.head()

,text,label
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드2
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드2
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드2
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드2


In [7]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}

train = train.replace({'label' : label_dict}).copy()

In [8]:
import re 

def clean_text(texts): 
    corpus = [] 
    for i in range(0, len(texts)): 

        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\>\<]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거 ]\[\은 예외처리
        review = re.sub(r'\d+','', review)#숫자 제거
        review = review.lower() #소문자 변환
        review = re.sub(r'\s+', ' ', review) #extra space 제거
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        review = re.sub(r'\s+', ' ', review) #spaces 제거
        review = re.sub(r"^\s+", '', review) #space from start 제거
        review = re.sub(r'\s+$', '', review) #space from the end 제거
        review = re.sub(r'_', ' ', review) #space from the end 제거
        corpus.append(review) 
        
    return corpus

In [9]:
temp = clean_text(train['text']) #메소드 적용
train['text'] = temp
train.head()

,text,label
0,selfconvs nnmodulelist[nnconvd co k for k in k...,0
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,1
2,globglobpath 를 사용할 때 질문입니다path에 [ ] 가 포함되면 제대로...,0
3,tmpp tmpgroupbyby addr asindexfalse[[category]...,0
4,filename testimage strroundframesec jpg이렇게 하면 ...,0


In [10]:
temp = clean_text(test['text']) #메소드 적용
test['text'] = temp
test.head()

,id,text
0,0,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요 이 경...
1,1,셀프테스트에서 받은 점수가 해당 교육을 이수하고 취업 연계등을 하는데 있어서 영향을...
2,2,a intinput정수 a의 값을 입력하시오b intinput정수 b의 값을 입력하...
3,3,def maxabcd maximum a if bgtmaximum maximumb i...
4,4,i 가 i 와 왜 같은지가 이해가 잘 안됩니다


In [11]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.1)
print(len(train_dataset))
print(len(val_dataset))

3335
371


In [12]:
# dataloader에서 오류가 나서 인덱스 재설정
train_dataset = train_dataset.reset_index(drop=True)
val_dataset = val_dataset.reset_index(drop=True)

In [13]:
# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        sentence1 = self.df_data.loc[index, 'text']

        encoded_dict = tokenizer.encode_plus(
                    sentence1,
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'label'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)
    
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        sentence1 = self.df_data.loc[index, 'text']


        encoded_dict = tokenizer.encode_plus(
                    sentence1, 
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)

In [14]:
# 모델 하이퍼파라미터

NUM_EPOCHS = 20
L_RATE = 1e-5 
MAX_LEN = 512

TRAIN_BATCH_SIZE = 4 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
TEST_BATCH_SIZE = 1

NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다. 

NUM_CORES

4

In [15]:
train_data = CompDataset(train_dataset)
val_data = CompDataset(val_dataset)
test_data = TestDataset(test)

# batch_size 만큼 데이터 분할
train_dataloader = DataLoader(train_data,
                                batch_size=TRAIN_BATCH_SIZE,
                                shuffle=True,
                                num_workers=NUM_CORES)

val_dataloader = DataLoader(val_data,
                            batch_size=TRAIN_BATCH_SIZE,
                            shuffle=True,
                            num_workers=NUM_CORES)

test_dataloader = DataLoader(test_data,
                                batch_size=TEST_BATCH_SIZE,
                                shuffle=False,
                                num_workers=NUM_CORES)



print(len(train_dataloader))
print(len(val_dataloader))
#print(len(test_dataloader))

834
93


In [16]:
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels = 5, # 출력 label의 개수
)

# model을 device위에 올림
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [17]:
# optimizer 설정
optimizer = AdamW(model.parameters(),
              lr = L_RATE, 
              eps = 1e-8 
            )

In [18]:
loss_fn = torch.nn.CrossEntropyLoss()

# loss값 저장
loss_values = []

# 학습 시작
for epoch in range(NUM_EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    
    stacked_val_labels = []
    targets_list = []

    # ========================================
    #               Training
    # ========================================
    
    print('Training...')
    
    # train mode 변환
    model.train()
    # True로 설정하게 되면 해당 텐서에서 어떤 연산이 이루어졌는지 추적할 수 있고, 해당 텐서에 대한 그라디언트를 저장하게 됩니다. 
    torch.set_grad_enabled(True)


    # 1epoch마다 loss값 초기화
    total_train_loss = 0

    for i, batch in enumerate(tqdm(train_dataloader)):
        
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        
        print(train_status, end='\r')


        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # 3개의 인풋
        outputs = model(b_input_ids, 
                    attention_mask=b_input_mask)
        
        # outputs tuple: (loss, logits)
        loss = loss_fn(outputs[0], b_labels)
        
        # loss는 텐서이기 때문에 숫자로 변환 후 더합니다. 
        total_train_loss = total_train_loss + loss.item()
        
        # backward()를 하기 전에 optimizer의 그라디언트를 0으로 합니다. 
        optimizer.zero_grad()
        
        # 그라디언트 계산
        loss.backward()
        
        
        # "exploding gradients" 문제를 예방해줍니다.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        
        # optimizer 가중치 업데이트
        optimizer.step() 
    
    print('Train loss:' ,total_train_loss)


    # ========================================
    #               Validation
    # ========================================
    
    print('\nValidation...')

    # evaluation mode로 변환
    model.eval()

    # validation 과정에서는 그라디언트를 연산하거나 저장하지 않습니다.(메모리, 진행 속도 세이브)
    torch.set_grad_enabled(False)
    
    total_val_loss = 0
    

    for j, batch in enumerate(tqdm(val_dataloader)):
        
        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)      


        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        # 예측값
        preds = outputs[0]

        loss = loss_fn(preds, b_labels)
        total_val_loss = total_val_loss + loss.item()
        

        # 예측값을 CPU로 이동시킵니다. 
        val_preds = preds.detach().cpu().numpy()
        
        # labels을 cpu로 이동시킵니다.
        targets_np = b_labels.to('cpu').numpy()

        targets_list.extend(targets_np)

        if j == 0:  # 첫 번째 batch일 떄
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))
        

    
    # validation accuracy 계산
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)
    
    val_acc = accuracy_score(y_true, y_pred)
    
    
    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)


    # 모델 저장
    torch.save(model.state_dict(), 'epoch:{}_model.pt'.format(epoch))
    
    # 메모리 관리
    gc.collect()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Val loss: 221.98762597795576
# Val acc:  0.6991341991341992

# Val loss: 248.69036069093272
# Val acc:  0.724025974025974

# Val loss: 298.04990690317936
# Val acc:  0.7337662337662337

In [22]:
# test 시작
for j, batch in enumerate(tqdm(test_dataloader)):
        
        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(test_dataloader))
        
        print(inference_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)

        outputs = model(b_input_ids, 
                attention_mask=b_input_mask)
        
        # 예측값
        preds = outputs[0]
        preds = preds.detach().cpu().numpy()

        
        if j == 0:  
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))

  0%|          | 3/929 [00:00<01:24, 10.98it/s]

  1%|          | 10/929 [00:00<00:41, 22.17it/s]

  2%|▏         | 17/929 [00:00<00:34, 26.75it/s]

  3%|▎         | 24/929 [00:01<00:31, 28.56it/s]

  3%|▎         | 28/929 [00:01<00:30, 29.30it/s]

...

 97%|█████████▋| 905/929 [00:30<00:00, 29.56it/s]

 98%|█████████▊| 911/929 [00:30<00:00, 29.42it/s]

 99%|█████████▊| 917/929 [00:30<00:00, 29.43it/s]

100%|█████████▉| 926/929 [00:31<00:00, 29.60it/s]

100%|██████████| 929/929 [00:31<00:00, 29.72it/s]

In [23]:
stacked_preds

array([[-3.0459113 , -1.8505572 , -1.7807115 ,  9.19453   , -2.9874454 ],
       [-2.8215895 , -2.198231  , -0.34101334,  8.424975  , -3.9297414 ],
       [ 8.990889  , -1.8008112 , -0.9828696 , -3.335078  , -3.1432462 ],
       ...,
       [ 8.848715  , -2.001413  , -0.8488461 , -2.3388834 , -3.7672305 ],
       [-2.6608174 ,  9.625731  , -2.0947766 , -2.6929874 , -2.7434402 ],
       [ 8.403111  ,  0.45563614, -1.9303375 , -2.5710566 , -4.2817125 ]],
      dtype=float32)

In [24]:
# argmax로 label결정
preds = np.argmax(stacked_preds, axis=1)

preds

array([3, 3, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 3, 0, 1, 0, 3, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 3, 0, 2, 2, 0, 0, 2, 3, 0, 4, 4, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3,
       3, 2, 0, 3, 2, 0, 4, 0, 4, 1, 0, 0, 2, 1, 1, 3, 3, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 1, 1, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 2, 1, 0, 0, 2, 0, 2, 0, 2, 1, 1, 2, 3, 3, 0, 0, 3, 2,
       2, 2, 2, 2, 1, 2, 2, 0, 0, 3, 2, 0, 0, 3, 0, 3, 0, 0, 3, 0, 1, 0,
       0, 4, 0, 2, 3, 0, 3, 2, 2, 2, 2, 3, 1, 1, 2, 2, 2, 3, 0, 2, 2, 3,
       3, 0, 2, 3, 0, 0, 2, 0, 0, 0, 4, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 4, 0,
       4, 1, 0, 0, 0, 0, 0, 2, 0, 0, 1, 3, 0, 0, 4, 3, 2, 0, 0, 0, 0, 0,
       0, 0, 1, 3, 3, 2, 0, 3, 3, 0, 3, 0, 3, 0, 0,

In [25]:
submission['label']=preds

In [26]:
submission.to_csv("sample_submission_5.csv", index = False)

In [ ]:
# 파일 저장
from google.colab import files
files.download("sample_submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>